In [37]:
from scipy.io import loadmat
import os
import numpy as np
import pandas as pd
from imblearn.ensemble import RUSBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
Data_path = 'D:\Classifier\Calssification2022_JZ\Code&Dataset\Week13_Classification_Final\Preprocessing'
Data_name = os.listdir(Data_path)
Data_name = [p for p in Data_name if 'mat' in p.split('.')]
Data_name = [p for p in Data_name if ('Site1' not in p.split('_'))&('Site2' not in p.split('_'))]

## 0: veh, 1:ped, 2:bike, 3:wheelchair 

In [42]:
Indicators = {
    'VehData':0,
    'PedData':1,
    'BikeData':2,
    'WheelchairData':3
}
column_names = ['TrajID','Mean_dis','Point_count','Dir','Height','Var_height','Len','Area','Label']
Train_Data = []
for i in range(len(Data_name)):
    data = loadmat(os.path.join(Data_path,Data_name[i]))
    class_name = list(data.keys())[3]
    data = data[class_name]
    class_ind = Indicators[class_name]
    class_col = (np.ones(len(data))*class_ind).reshape(-1,1).astype('int')
    try:
        data = np.concatenate([data[:,-3].reshape(-1,1).astype(int),data[:,:7],class_col],axis = 1)
        data = pd.DataFrame(data,columns=column_names)
    except:
        continue
    Train_Data.append(data)

In [52]:
Valid_train_data = []
for Trajs in Train_Data:
    for ind,traj in Trajs.groupby('TrajID'):
        if ind == -1:
            continue
        if len(traj) > 20:
            traj.Len = traj.Len.max()
            Valid_train_data.append(traj)

In [53]:
Valid_train_data = pd.concat(Valid_train_data)

In [54]:
X,y = np.array(Valid_train_data.iloc[:,1:-1]),np.array(Valid_train_data.iloc[:,-1])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [56]:
rusboost = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R',
                              random_state=0)
rusboost.fit(X_train, y_train) 

RUSBoostClassifier(n_estimators=200, random_state=0)

In [57]:
y_pred = rusboost.predict(X_test)

In [58]:
confusion_matrix(y_test, y_pred,normalize = 'true')

array([[9.98806376e-01, 0.00000000e+00, 1.15308563e-03, 4.05381667e-05],
       [2.40905806e-04, 8.38593110e-01, 1.27439171e-01, 3.37268128e-02],
       [8.33614808e-02, 1.24974667e-01, 6.87090455e-01, 1.04573397e-01],
       [6.83060109e-03, 3.63387978e-01, 3.14207650e-01, 3.15573770e-01]])

In [59]:
balanced_accuracy_score(y_test, y_pred)  

0.7100159278566314

In [231]:
path = 'D:\Classifier\Calssification2022_JZ\Code&Dataset\ThreeCasesinPaper\Case 2_TrajectoryGreaterThan20Frames\ClassificationConsideringHistoricalTrajectory\RUSBoost'

In [232]:
X_train = loadmat(os.path.join(path,'Train_Data.mat'))['Train_Data']
y_train = loadmat(os.path.join(path,'Train_Target.mat'))['Train_Target']
y_train = y_train.flatten()
X_test = loadmat(os.path.join(path,'Test_Data.mat'))['Test_Data']
y_test = loadmat(os.path.join(path,'Test_Target.mat'))['Test_Target']
y_test = y_test.flatten()
ind = [i for i in range(7) if i!= 4]
X_train = X_train[:,ind] 
X_test = X_test[:,ind] 

In [330]:
X_test[:,2] = np.zeros(len(X_test))

In [325]:
rusboost = RUSBoostClassifier(n_estimators=8, algorithm='SAMME.R',
                              random_state=0)
rusboost.fit(X_train, y_train) 

RUSBoostClassifier(n_estimators=8, random_state=0)

In [331]:
y_pred = rusboost.predict(X_test)

In [332]:
confusion_matrix(y_test, y_pred)

array([[2904,    0,    1,    0],
       [   0, 2379,  193,   83],
       [   0,   24,  292,   70],
       [   0,   47,   72,  588]], dtype=int64)

In [333]:
balanced_accuracy_score(y_test, y_pred)  

0.8709652039788986